In [1]:
import sys
import os
from accelerate import notebook_launcher
from datasets import load_dataset
import logging

# Configure logging:
# (i) lear any pre-existing logging handlers (especially useful in notebook reruns)
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
# (ii) now safely configure logging
logging.getLogger("codecarbon").setLevel(logging.ERROR)
logging.basicConfig(level=logging.INFO)

# Adjust paths -> import classes
project_root = os.getcwd()  
if project_root not in sys.path:
    sys.path.append(project_root)
from classes.experiment_config import ExperimentConfig
from classes.experiment_runner import ExperimentRunner

#import torch.multiprocessing as mp
#mp.set_start_method('spawn', force=True)

In [ ]:
experiment_config = ExperimentConfig(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    is_encoder_decoder=True,
    task_type="text_generation",
    inference_type="pure_generative",  
    max_input_tokens=128,
    max_output_tokens=128,
    num_input_prompts=50,
    save_outputs=False,
    decode_token_to_text=True,
    gpu_list=[0,1,2,3],
    num_processes=4,
    batching_options={
        "fixed_max_batch_size": 8, # this is max batch size if adaptive batching on; fixed batch size if it's off
        "adaptive_batching": False,
        "adaptive_max_tokens": 512  
    },
    sharding_config={
        "fsdp_config": {
            "use_orig_params": True,
            "cpu_offload": True
        },
        "sharding_strategy": "NO_SHARD"
    },
    query_rate=1,
    decoder_temperature=1.0, # NB: needs to be a float
    fp_precision="float16",
    backend="pytorch"
)

# Load prompts (here, using the 'arxiv' split from the lighteval/pile_helm dataset)
ds = load_dataset("lighteval/pile_helm", "arxiv")["test"]
prompts = [sample["text"] for sample in ds]

# function to run the whole experiment workflow.
def run_experiment_workflow():
    # run experiment
    runner = ExperimentRunner(experiment_config, prompts)
    runner.run_torch()
    
    # If not main process, exit cleanly after run
    if not runner.accelerator.is_main_process:
        return  # do NOT proceed further

    # Only main process continues
    if runner.accelerator.is_main_process:
         # aggregate results
        runner.aggregate_results()
        # save results
        runner.save_experiment_results()  
        
    # could add in script here to delete JSON files if storage an issue
    
    runner.teardown()

    return

# Launch the experiment across the specified number of processes.
notebook_launcher(run_experiment_workflow,
                    num_processes=experiment_config.num_processes) 
                    #terminate_on_error=True) # USE THIS IN accelerate.launch.launch()                 

Launching training on 4 GPUs.
Accelerator set up
Unique experiment id: 0191
TinyLlama/TinyLlama-1.1B-Chat-v1.0 loaded using pytorch, with precision float16
Original generate method saved.
Model and tokenizer prepared
Entering wait barrier: after model preparation
Exiting wait barrier: after model preparation


INFO:classes.experiment_runner:[Process 2648911] Model is on device: cuda:1
INFO:classes.experiment_runner:[Process 2648910] Model is on device: cuda:0


Entering wait barrier: after logging device info

INFO:classes.experiment_runner:[Process 2648912] Model is on device: cuda:2
INFO:classes.experiment_runner:[Process 2648913] Model is on device: cuda:3



Exiting wait barrier: after logging device info
Original generate method reassigned


...

INFO:classes.experiment_runner:[Process 2648912] Dummy forward pass complete
INFO:classes.experiment_runner:[Process 2648913] Dummy forward pass complete


Exiting wait barrier: after dummy forward pass
Prompts processed: 100 prompts.


INFO:_6_run_inference_by_task:[Process 2648911][GPU 1] — Completed tokenisation of batch 1/13


Energy tracking started
Task type: pure_generative
Using fixed batching (non-adaptive): created 13 batches.


INFO:_6_run_inference_by_task:[Process 2648913][GPU 3] — Completed tokenisation of batch 1/13
INFO:_6_run_inference_by_task:[Process 2648912][GPU 2] — Completed tokenisation of batch 1/13
INFO:_6_run_inference_by_task:[Process 2648910][GPU 0] — Completed tokenisation of batch 1/13
INFO:_6_run_inference_by_task:[Process 2648913][GPU 3] — Completed batch inference 1/13
INFO:_6_run_inference_by_task:[Process 2648913][GPU 3] — Completed tokenisation of batch 2/13
INFO:_6_run_inference_by_task:[Process 2648911][GPU 1] — Completed batch inference 1/13
INFO:_6_run_inference_by_task:[Process 2648911][GPU 1] — Completed tokenisation of batch 2/13
INFO:_6_run_inference_by_task:[Process 2648910][GPU 0] — Completed batch inference 1/13
INFO:_6_run_inference_by_task:[Process 2648910][GPU 0] — Completed tokenisation of batch 2/13
INFO:_6_run_inference_by_task:[Process 2648912][GPU 2] — Completed batch inference 1/13
INFO:_6_run_inference_by_task:[Process 2648912][GPU 2] — Completed tokenisation of b